In [6]:
import pandas as pd
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import re



In [7]:
file_path = 'data.csv'
df = pd.read_csv(file_path)
print("Initial Data:")
display(df.head())

Initial Data:


,id,source,title,image,url,content,date,embedding,created_at,updated_at,summary
0,83,tempo,"Depo Plumpang Terbakar, Anggota DPR Minta Pert...",https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698528/depo-pl...,"TEMPO.CO, Jakarta - Anggota Komisi VII DPR RI ...",2023-03-04 06:18:13+00,"[-0.01590039,-0.034130897,0.005732614,-0.01853...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Anggota Komisi VII DPR RI Rofik Hananto menyay...
1,84,tempo,Jokowi Perintahkan Wapres Ma'ruf Amin Tinjau L...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698522/jokowi-...,"TEMPO.CO, Jakarta - Presiden Joko Widodo atau ...",2023-03-04 06:04:38+00,"[-0.017608976,-0.021786924,0.01547983,-0.00932...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Presiden Joko Widodo telah memerintahkan Wakil...
2,85,tempo,HNW Mendukung Jamaah Umroh First Travel Dapatk...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698527/hnw-men...,INFO NASIONAL - Wakil Ketua MPR RI Dr. H. M. H...,2023-03-04 06:18:04+00,"[0.00841488,-0.023665192,0.006762431,-0.013723...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Wakil Ketua MPR RI Dr. H. M. Hidayat Nur Wahid...
3,86,tempo,Tim Dokkes Polri Telah Terima 14 Kantong Jenaz...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698540/tim-dok...,"TEMPO.CO, Jakarta - Tim Kedokteran dan Kesehat...",2023-03-04 06:44:10+00,"[-0.012671886,-0.0039057182,0.019575326,-0.016...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Tim Kedokteran dan Kesehatan (Dokkes) Polri te...
4,87,tempo,Bamsoet Ajak Komunitas Otomotif Kembangkan Per...,https://statik.tempo.co/data/2023/03/04/id_118...,https://nasional.tempo.co/read/1698536/bamsoet...,INFO NASIONAL - Ketua MPR RI sekaligus Ketua U...,2023-03-04 06:38:57+00,"[-0.015486176,-0.0125719,-0.0122843925,-0.0343...",2023-03-04 07:03:39.039332,2023-03-04 07:03:39.039332,Ketua MPR RI Bambang Soesatyo telah diangkat s...


In [8]:
columns_to_drop = ['image', 'url', 'embedding', 'created_at', 'updated_at']
df.drop(columns=columns_to_drop, inplace=True)
print("\nAfter Dropping Unnecessary Columns:")
display(df.head())


After Dropping Unnecessary Columns:


,id,source,title,content,date,summary
0,83,tempo,"Depo Plumpang Terbakar, Anggota DPR Minta Pert...","TEMPO.CO, Jakarta - Anggota Komisi VII DPR RI ...",2023-03-04 06:18:13+00,Anggota Komisi VII DPR RI Rofik Hananto menyay...
1,84,tempo,Jokowi Perintahkan Wapres Ma'ruf Amin Tinjau L...,"TEMPO.CO, Jakarta - Presiden Joko Widodo atau ...",2023-03-04 06:04:38+00,Presiden Joko Widodo telah memerintahkan Wakil...
2,85,tempo,HNW Mendukung Jamaah Umroh First Travel Dapatk...,INFO NASIONAL - Wakil Ketua MPR RI Dr. H. M. H...,2023-03-04 06:18:04+00,Wakil Ketua MPR RI Dr. H. M. Hidayat Nur Wahid...
3,86,tempo,Tim Dokkes Polri Telah Terima 14 Kantong Jenaz...,"TEMPO.CO, Jakarta - Tim Kedokteran dan Kesehat...",2023-03-04 06:44:10+00,Tim Kedokteran dan Kesehatan (Dokkes) Polri te...
4,87,tempo,Bamsoet Ajak Komunitas Otomotif Kembangkan Per...,INFO NASIONAL - Ketua MPR RI sekaligus Ketua U...,2023-03-04 06:38:57+00,Ketua MPR RI Bambang Soesatyo telah diangkat s...


In [9]:
columns_rename_map = {
    'id': 'record_id',
    'source': 'news_source',
    'date': 'publication_date'
}
df.rename(columns=columns_rename_map, inplace=True)
print("\nAfter Renaming Columns:")
display(df.head())



After Renaming Columns:


,record_id,news_source,title,content,publication_date,summary
0,83,tempo,"Depo Plumpang Terbakar, Anggota DPR Minta Pert...","TEMPO.CO, Jakarta - Anggota Komisi VII DPR RI ...",2023-03-04 06:18:13+00,Anggota Komisi VII DPR RI Rofik Hananto menyay...
1,84,tempo,Jokowi Perintahkan Wapres Ma'ruf Amin Tinjau L...,"TEMPO.CO, Jakarta - Presiden Joko Widodo atau ...",2023-03-04 06:04:38+00,Presiden Joko Widodo telah memerintahkan Wakil...
2,85,tempo,HNW Mendukung Jamaah Umroh First Travel Dapatk...,INFO NASIONAL - Wakil Ketua MPR RI Dr. H. M. H...,2023-03-04 06:18:04+00,Wakil Ketua MPR RI Dr. H. M. Hidayat Nur Wahid...
3,86,tempo,Tim Dokkes Polri Telah Terima 14 Kantong Jenaz...,"TEMPO.CO, Jakarta - Tim Kedokteran dan Kesehat...",2023-03-04 06:44:10+00,Tim Kedokteran dan Kesehatan (Dokkes) Polri te...
4,87,tempo,Bamsoet Ajak Komunitas Otomotif Kembangkan Per...,INFO NASIONAL - Ketua MPR RI sekaligus Ketua U...,2023-03-04 06:38:57+00,Ketua MPR RI Bambang Soesatyo telah diangkat s...


In [11]:
df.head()

,record_id,news_source,title,content,publication_date,summary
0,83,tempo,"depo plumpang terbakar, anggota dpr minta pert...","tempo.co, jakarta - anggota komisi 7 dpr ri ro...",2023-03-04 06:18:13+00,anggota komisi 7 dpr ri rofik hananto menyayan...
1,84,tempo,jokowi perintahkan wapres ma'ruf amin tinjau l...,"tempo.co, jakarta - presiden joko widodo atau ...",2023-03-04 06:04:38+00,presiden joko widodo telah memerintahkan wakil...
2,85,tempo,hnw mendukung jamaah umroh first travel dapatk...,info nasional - wakil ketua mpr ri dr. h. m. h...,2023-03-04 06:18:04+00,wakil ketua mpr ri dr. h. m. hidayat nur wahid...
3,86,tempo,tim dokkes polri telah terima 14 kantong jenaz...,"tempo.co, jakarta - tim kedokteran dan kesehat...",2023-03-04 06:44:10+00,tim kedokteran dan kesehatan (dokkes) polri te...
4,87,tempo,bamsoet ajak komunitas otomotif kembangkan per...,info nasional - ketua mpr ri sekaligus ketua u...,2023-03-04 06:38:57+00,ketua mpr ri bambang soesatyo telah diangkat s...


In [10]:
def roman_to_int(roman):
    roman_map = {
        'I': 1, 'II': 2, 'III': 3, 'IV': 4, 
        'V': 5, 'VI': 6, 'VII': 7, 'VIII': 8, 
        'IX': 9, 'X': 10
    }

    return roman_map.get(roman, roman)


def convert_roman_to_int(text):
    def replace_roman(match):
        roman_numeral = match.group(0)
        return str(roman_to_int(roman_numeral))  # Convert Roman numeral to integer
    

    return re.sub(r'\b(I|II|III|IV|V|VI|VII|VIII|IX|X)\b', replace_roman, text)


for col in ['title', 'content', 'summary']:
    df[col] = df[col].apply(lambda x: convert_roman_to_int(x) if isinstance(x, str) else x)
    df[col] = df[col].str.lower()  # Convert to lowercase


display(df.head())


,record_id,news_source,title,content,publication_date,summary
0,83,tempo,"depo plumpang terbakar, anggota dpr minta pert...","tempo.co, jakarta - anggota komisi 7 dpr ri ro...",2023-03-04 06:18:13+00,anggota komisi 7 dpr ri rofik hananto menyayan...
1,84,tempo,jokowi perintahkan wapres ma'ruf amin tinjau l...,"tempo.co, jakarta - presiden joko widodo atau ...",2023-03-04 06:04:38+00,presiden joko widodo telah memerintahkan wakil...
2,85,tempo,hnw mendukung jamaah umroh first travel dapatk...,info nasional - wakil ketua mpr ri dr. h. m. h...,2023-03-04 06:18:04+00,wakil ketua mpr ri dr. h. m. hidayat nur wahid...
3,86,tempo,tim dokkes polri telah terima 14 kantong jenaz...,"tempo.co, jakarta - tim kedokteran dan kesehat...",2023-03-04 06:44:10+00,tim kedokteran dan kesehatan (dokkes) polri te...
4,87,tempo,bamsoet ajak komunitas otomotif kembangkan per...,info nasional - ketua mpr ri sekaligus ketua u...,2023-03-04 06:38:57+00,ketua mpr ri bambang soesatyo telah diangkat s...


In [ ]:
df['Status'] = 1

print(df)

       record_id news_source  \
0             83       tempo   
1             84       tempo   
2             85       tempo   
3             86       tempo   
4             87       tempo   
...          ...         ...   
32730      63563    kumparan   
32731      63564    kumparan   
32732      63565    kumparan   
32733      63566    kumparan   
32734      63567       tempo   

                                                   title  \
0      depo plumpang terbakar, anggota dpr minta pert...   
1      jokowi perintahkan wapres ma'ruf amin tinjau l...   
2      hnw mendukung jamaah umroh first travel dapatk...   
3      tim dokkes polri telah terima 14 kantong jenaz...   
4      bamsoet ajak komunitas otomotif kembangkan per...   
...                                                  ...   
32730  5 hewan aneh yang hidup di palung mariana, sal...   
32731  biadab! guru sd di banyuwangi cabuli siswinya,...   
32732  aji santoso: jakmania perlakukan persebaya den...   
32733  sopir au

In [15]:
cleaned_file_path = 'cleaned_data.csv'
df.to_csv(cleaned_file_path, index=False)
print(f"\nCleaned data saved to {cleaned_file_path}")
df.tail()


Cleaned data saved to cleaned_data.csv


,record_id,news_source,title,content,publication_date,summary,Status
32730,63563,kumparan,"5 hewan aneh yang hidup di palung mariana, sal...",menjadi salah satu lubang terdalam dan tergela...,2023-04-11 20:13:25+00,peneliti berhasil mengungkap sedikit demi sedi...,1
32731,63564,kumparan,"biadab! guru sd di banyuwangi cabuli siswinya,...",seorang guru sd di banyuwangi ditangkap oleh p...,2023-04-11 20:05:30+00,seorang guru sd di banyuwangi ditangkap polisi...,1
32732,63565,kumparan,aji santoso: jakmania perlakukan persebaya den...,"pelatih , aji santoso, salut dengan sambutan s...",2023-04-11 19:37:49+00,"pelatih persebaya, aji santoso, menyampaikan s...",1
32733,63566,kumparan,sopir audi cium kaki ibu & istri hamil usai si...,suasana haru terjadi usai sidang terdakwa suge...,2023-04-11 19:24:07+00,"sugeng guruh gautama legiman, terdakwa kasus k...",1
32734,63567,tempo,ini besaran biaya haji 2023 per embarkasi,"tempo.co, jakarta - presiden joko widodo telah...",2023-04-11 20:00:00+00,presiden jokowi telah menetapkan besaran biaya...,1


In [16]:

cleaned_file_path = 'cleaned_data.csv'
df = pd.read_csv(cleaned_file_path)

columns_to_tokenize = ['title', 'content', 'summary']

tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

for column in columns_to_tokenize:
    df[f"{column}_tokens"] = df[column].apply(
        lambda x: tokenizer.tokenize(x) if pd.notnull(x) else []
    )
    df[f"{column}_token_ids"] = df[column].apply(
        lambda x: tokenizer.encode(x, truncation=True, max_length=512) if pd.notnull(x) else []
    )

tokenized_file_path = 'tokenized_data.csv'
df.to_csv(tokenized_file_path, index=False)
print(f"Tokenized data saved to {tokenized_file_path}")


Tokenized data saved to tokenized_data.csv


In [18]:
tokenized_df = pd.read_csv('tokenized_data.csv')
tokenized_df2= pd.read_csv('tokenized_data_hoax.csv')
tokenized_df2.head()


,record_id,title,content,publication_date,Status,title_tokens,title_token_ids,content_tokens,content_token_ids
0,495,foto jim rohn: jokowi adalah presiden terbaik ...,jokowi adalah presiden terbaik dlm sejarah ban...,13-jul-20,0,"['f', 'oto', 'Ġj', 'im', 'Ġro', 'hn', ':', 'Ġj...","[0, 506, 7087, 1236, 757, 4533, 10245, 35, 123...","['j', 'ok', 'ow', 'i', 'Ġad', 'al', 'ah', 'Ġpr...","[0, 267, 1638, 1722, 118, 2329, 337, 895, 9383..."
1,550,"ini bukan politik, tapi kenyataan pak jokowi b...","maaf mas2 dan mbak2, ini bukan politik, tapi k...",8-jul-20,0,"['ini', 'Ġbu', 'kan', 'Ġpolit', 'ik', ',', 'Ġt...","[0, 2531, 10306, 11334, 10635, 967, 6, 5797, 1...","['ma', 'af', 'Ġmas', '2', 'Ġdan', 'Ġm', 'b', '...","[0, 1916, 2001, 11705, 176, 20435, 475, 428, 6..."
2,681,foto kadrun kalo lihat foto ini panas dingin,kadrun kalo lihat foto ini panas dingin . .,24-jun-20,0,"['f', 'oto', 'Ġk', 'ad', 'run', 'Ġk', 'alo', '...","[0, 506, 7087, 449, 625, 2962, 449, 7747, 784,...","['k', 'ad', 'run', 'Ġk', 'alo', 'Ġl', 'ih', 'a...","[0, 330, 625, 2962, 449, 7747, 784, 4001, 415,..."
3,736,event promo smartphone jne 2020 spesial di bul...,"selamat siang teman teman fb ku semuanya🤩,cuma...",17-jun-20,0,"['event', 'Ġpromo', 'Ġsmartphone', 'Ġj', 'ne',...","[0, 21680, 21316, 4368, 1236, 858, 2760, 2292,...","['sel', 'am', 'at', 'Ġs', 'iang', 'Ġtem', 'an'...","[0, 5317, 424, 415, 579, 14607, 11030, 260, 11..."
4,1161,mereka sudah mempersiapkan diri dengan baik un...,lihatlah mereka sudah mempersiapkan diri denga...,5-may-20,0,"['mere', 'ka', 'Ġsu', 'd', 'ah', 'Ġmem', 'pers...","[0, 25416, 2348, 2628, 417, 895, 26012, 16414,...","['l', 'ih', 'at', 'lah', 'Ġmere', 'ka', 'Ġsu',...","[0, 462, 4001, 415, 30166, 9198, 2348, 2628, 4..."


In [22]:
tokenized_df.head()

,record_id,news_source,title,content,publication_date,summary,Status,title_tokens,title_token_ids,content_tokens,content_token_ids,summary_tokens,summary_token_ids
0,83,tempo,"depo plumpang terbakar, anggota dpr minta pert...","tempo.co, jakarta - anggota komisi 7 dpr ri ro...",2023-03-04 06:18:13+00,anggota komisi 7 dpr ri rofik hananto menyayan...,1,"['dep', 'o', 'Ġpl', 'ump', 'ang', 'Ġter', 'b',...","[0, 17272, 139, 2968, 7198, 1097, 8470, 428, 6...","['tem', 'po', '.', 'co', ',', 'Ġj', 'ak', 'art...","[0, 17950, 5873, 4, 876, 6, 1236, 677, 28721, ...","['ang', 'g', 'ota', 'Ġk', 'om', 'isi', 'Ġ7', '...","[0, 1097, 571, 6804, 449, 1075, 8539, 262, 385..."
1,84,tempo,jokowi perintahkan wapres ma'ruf amin tinjau l...,"tempo.co, jakarta - presiden joko widodo atau ...",2023-03-04 06:04:38+00,presiden joko widodo telah memerintahkan wakil...,1,"['j', 'ok', 'ow', 'i', 'Ġper', 'int', 'ah', 'k...","[0, 267, 1638, 1722, 118, 228, 2544, 895, 1133...","['tem', 'po', '.', 'co', ',', 'Ġj', 'ak', 'art...","[0, 17950, 5873, 4, 876, 6, 1236, 677, 28721, ...","['pres', 'iden', 'Ġj', 'oko', 'Ġwid', 'odo', '...","[0, 13529, 12145, 1236, 11136, 23772, 15537, 1..."
2,85,tempo,hnw mendukung jamaah umroh first travel dapatk...,info nasional - wakil ketua mpr ri dr. h. m. h...,2023-03-04 06:18:04+00,wakil ketua mpr ri dr. h. m. hidayat nur wahid...,1,"['hn', 'w', 'Ġmend', 'uk', 'ung', 'Ġj', 'ama',...","[0, 10245, 605, 19300, 1350, 1545, 1236, 2583,...","['info', 'Ġnas', 'ional', 'Ġ-', 'Ġw', 'ak', 'i...","[0, 23999, 23473, 6073, 111, 885, 677, 718, 21...","['w', 'ak', 'il', 'Ġket', 'ua', 'Ġm', 'pr', 'Ġ...","[0, 605, 677, 718, 21095, 4324, 475, 4862, 910..."
3,86,tempo,tim dokkes polri telah terima 14 kantong jenaz...,"tempo.co, jakarta - tim kedokteran dan kesehat...",2023-03-04 06:44:10+00,tim kedokteran dan kesehatan (dokkes) polri te...,1,"['tim', 'Ġdo', 'k', 'kes', 'Ġpol', 'ri', 'Ġtel...","[0, 10519, 109, 330, 7339, 8385, 1069, 19676, ...","['tem', 'po', '.', 'co', ',', 'Ġj', 'ak', 'art...","[0, 17950, 5873, 4, 876, 6, 1236, 677, 28721, ...","['tim', 'Ġk', 'ed', 'ok', 'ter', 'an', 'Ġdan',...","[0, 10519, 449, 196, 1638, 1334, 260, 20435, 4..."
4,87,tempo,bamsoet ajak komunitas otomotif kembangkan per...,info nasional - ketua mpr ri sekaligus ketua u...,2023-03-04 06:38:57+00,ketua mpr ri bambang soesatyo telah diangkat s...,1,"['b', 'am', 'so', 'et', 'Ġa', 'j', 'ak', 'Ġk',...","[0, 428, 424, 2527, 594, 10, 267, 677, 449, 10...","['info', 'Ġnas', 'ional', 'Ġ-', 'Ġket', 'ua', ...","[0, 23999, 23473, 6073, 111, 21095, 4324, 475,...","['ket', 'ua', 'Ġm', 'pr', 'Ġr', 'i', 'Ġb', 'am...","[0, 19613, 4324, 475, 4862, 910, 118, 741, 314..."


In [25]:
df_merge = pd.merge(tokenized_df, tokenized_df2, on=['record_id', 'content','Status','title','publication_date','title_token_ids','title_tokens','content_tokens','content_token_ids'], how='outer')

df_merge.head()


,record_id,news_source,title,content,publication_date,summary,Status,title_tokens,title_token_ids,content_tokens,content_token_ids,summary_tokens,summary_token_ids
0,83,tempo,"depo plumpang terbakar, anggota dpr minta pert...","tempo.co, jakarta - anggota komisi 7 dpr ri ro...",2023-03-04 06:18:13+00,anggota komisi 7 dpr ri rofik hananto menyayan...,1,"['dep', 'o', 'Ġpl', 'ump', 'ang', 'Ġter', 'b',...","[0, 17272, 139, 2968, 7198, 1097, 8470, 428, 6...","['tem', 'po', '.', 'co', ',', 'Ġj', 'ak', 'art...","[0, 17950, 5873, 4, 876, 6, 1236, 677, 28721, ...","['ang', 'g', 'ota', 'Ġk', 'om', 'isi', 'Ġ7', '...","[0, 1097, 571, 6804, 449, 1075, 8539, 262, 385..."
1,84,tempo,jokowi perintahkan wapres ma'ruf amin tinjau l...,"tempo.co, jakarta - presiden joko widodo atau ...",2023-03-04 06:04:38+00,presiden joko widodo telah memerintahkan wakil...,1,"['j', 'ok', 'ow', 'i', 'Ġper', 'int', 'ah', 'k...","[0, 267, 1638, 1722, 118, 228, 2544, 895, 1133...","['tem', 'po', '.', 'co', ',', 'Ġj', 'ak', 'art...","[0, 17950, 5873, 4, 876, 6, 1236, 677, 28721, ...","['pres', 'iden', 'Ġj', 'oko', 'Ġwid', 'odo', '...","[0, 13529, 12145, 1236, 11136, 23772, 15537, 1..."
2,85,tempo,hnw mendukung jamaah umroh first travel dapatk...,info nasional - wakil ketua mpr ri dr. h. m. h...,2023-03-04 06:18:04+00,wakil ketua mpr ri dr. h. m. hidayat nur wahid...,1,"['hn', 'w', 'Ġmend', 'uk', 'ung', 'Ġj', 'ama',...","[0, 10245, 605, 19300, 1350, 1545, 1236, 2583,...","['info', 'Ġnas', 'ional', 'Ġ-', 'Ġw', 'ak', 'i...","[0, 23999, 23473, 6073, 111, 885, 677, 718, 21...","['w', 'ak', 'il', 'Ġket', 'ua', 'Ġm', 'pr', 'Ġ...","[0, 605, 677, 718, 21095, 4324, 475, 4862, 910..."
3,86,tempo,tim dokkes polri telah terima 14 kantong jenaz...,"tempo.co, jakarta - tim kedokteran dan kesehat...",2023-03-04 06:44:10+00,tim kedokteran dan kesehatan (dokkes) polri te...,1,"['tim', 'Ġdo', 'k', 'kes', 'Ġpol', 'ri', 'Ġtel...","[0, 10519, 109, 330, 7339, 8385, 1069, 19676, ...","['tem', 'po', '.', 'co', ',', 'Ġj', 'ak', 'art...","[0, 17950, 5873, 4, 876, 6, 1236, 677, 28721, ...","['tim', 'Ġk', 'ed', 'ok', 'ter', 'an', 'Ġdan',...","[0, 10519, 449, 196, 1638, 1334, 260, 20435, 4..."
4,87,tempo,bamsoet ajak komunitas otomotif kembangkan per...,info nasional - ketua mpr ri sekaligus ketua u...,2023-03-04 06:38:57+00,ketua mpr ri bambang soesatyo telah diangkat s...,1,"['b', 'am', 'so', 'et', 'Ġa', 'j', 'ak', 'Ġk',...","[0, 428, 424, 2527, 594, 10, 267, 677, 449, 10...","['info', 'Ġnas', 'ional', 'Ġ-', 'Ġket', 'ua', ...","[0, 23999, 23473, 6073, 111, 21095, 4324, 475,...","['ket', 'ua', 'Ġm', 'pr', 'Ġr', 'i', 'Ġb', 'am...","[0, 19613, 4324, 475, 4862, 910, 118, 741, 314..."


In [13]:
model = SentenceTransformer("indonesian-sbert")

df['combined_text'] = df['title'] + " " + df['content'] + " " + df['summary']
text_embeddings = model.encode(df['combined_text'].tolist())


No sentence-transformers model found with name sentence-transformers/indonesian-sbert. Creating a new one with mean pooling.


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Raja\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\utils\_http.py", line 406, in hf_raise_for_status
    response.raise_for_status()
  File "c:\Users\Raja\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/sentence-transformers/indonesian-sbert/resolve/main/adapter_config.json

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\Raja\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\hub.py", line 403, in cached_file
    resolved_file = hf_hub_download(
  File "c:\Users\Raja\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\utils\_validators.py", line 114, in _inner_fn
    return fn(*args,